<a href="https://colab.research.google.com/github/EvenSol/NeqSim-Colab/blob/master/notebooks/energyopt/PyPSA_Norway_Atlite_PPM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PyPSA Norge med Atlite og PowerplantmatchingDette eksemplet viser hvordan du kan sette opp et enkelt PyPSA-nettverk for Norge i Google Colab.Vi henter værdata fra **atlite** og kraftverksdata fra **powerplantmatching**. Nettverksdata kan lastes ned fra NVE eller ENTSO-E og leses inn med `geopandas`.

In [ ]:
!pip install pypsa atlite powerplantmatching geopandas

In [ ]:
import atlite
cutout = atlite.Cutout('norway', module='era5', x=slice(4, 12), y=slice(57, 71), time='2021')
# Dette vil laste ned ERA5 data (kan ta litt tid)
cutout.prepare()

In [ ]:
from powerplantmatching import DataPortal
ppm = DataPortal.fetch('powerplantmatching')
plants_no = ppm.get_powerplants(country='Norway')
print(plants_no[['Name', 'Fueltype', 'Capacity']].head())

## Nettverksdata
Du kan hente inn transmisjonslinjer fra NVE sitt [åpne datalager](https://www.nve.no/energi/kraftsystemet/nettanlegg/) eller ENTSO-E. Filene kan leses med `geopandas` slik:


In [ ]:
import geopandas as gpd
# Example: gpd.read_file('path_to/NVE_Nettanlegg.shp')

In [ ]:
import pypsa

network = pypsa.Network()

# Legg til en buss for eksempelvis Oslo
network.add('Bus', 'Oslo')
# Legg til en generator med produksjonsprofil fra atlite-cutouten
profile = cutout.wind(turbine='Vestas_V90_3MW', panel=None)
network.add('Generator', 'WindFarm', bus='Oslo', p_nom=10, p_max_pu=profile)

network.snapshots = profile.index
print(network)